In [1]:
import os
import numpy as np
import glob
import json
import matplotlib.pyplot as plt
import gc
import pandas as pd
from baseband_analysis.analysis.toa import get_TOA
from baseband_analysis.core import BBData
from chime_frb_api import frb_master
import datetime as dt
from astropy.time import Time
gc.collect()

--------------------------------------------------------------------------
[[19961,1],0]: A high-performance Open MPI point-to-point messaging module
was unable to find any relevant network interfaces:

Module: OpenFabrics (openib)
  Host: 1bae6bb3b670

Another transport will be used instead, although this may result in
lower performance.

NOTE: You can disable this warning by setting the MCA parameter
btl_base_warn_component_unused to 0.
--------------------------------------------------------------------------


19

In [2]:
basepath = '/data/user-data/ksand/R3/fitburst_run_final/'
entries = os.listdir(basepath)
entries.remove('.ipynb_checkpoints')
entries = np.array(entries).astype(int)
entries = np.sort(entries)
print(entries)

[ 23789294  23891929  40363753  40750224  40750656  60073678  65395717
  65511496  67522832  67522901  69509443  71665813  71784400  71784833
  71785167  73726599  83984528  84081108  92773515  92844954 135577833
 151677988 153859596 154910364 154913824 155079679 163181014 163362088
 163534005 166812476 171681248 171855709 175004577 175929272 176060009
 180666196 180667327 188553320 194488374 194488393 194488454 195132396
 200973875 203798724 203935225]


In [3]:
basepath = '/data/user-data/ksand/R3/fitburst_run_final/'
event_ids = []
pm_files = []
result_files = []
input_files = []
summary = []
for event in entries:
    if (event != '.ipynb_checkpoints'): 
        path_to_event = os.path.join(basepath, str(event))
        trials = np.array(os.listdir(path_to_event))
#         print(event, len(trials))
        count = 0


        for trial in trials:
            if (trial != '.ipynb_checkpoints'):
                path_to_file = os.path.join(path_to_event, trial)
                #print(event, path_to_file, "\n")
                #print(path_to_file)
                event_ids.append(int(event))
                np.array(pm_files.append(glob.glob(path_to_file + '/pm*.npz')))
                #print(pm_files)
                np.array(result_files.append(glob.glob(path_to_file + '/result*.npz')))
                np.array(input_files.append(glob.glob(path_to_file + '/R3*.npz')))
                np.array(summary.append(glob.glob(path_to_file + '/summary*.png')))
                count = count + 1

In [4]:
DMs = []
downsamps = []
for pm_file in pm_files:
    if len(pm_file) > 0:
        data_pm = np.load(pm_file[0], allow_pickle = True)
        dm = data_pm['arr_0'].item(0)['DM']
        ds = data_pm['arr_0'].item(0)['ds']
        DMs.append(float(dm))
        downsamps.append(int(ds))
print(downsamps)

In [12]:
scatterings = []
widths = []
sc_error = []
width_err = []
dm_offset = []
dm_offset_err = []
event_id_neg = []
for result_file in result_files:
    #print(result_file)
    if len(result_file) > 0:
        data_result = np.load(result_file[0], allow_pickle = True)
        scatterings.append(float(data_result['arr_0'][0]['scattering_timescale'][0]))
        if (data_result['arr_0'][0]['scattering_timescale'][0]) < 0:
            print(result_file[0])
        widths.append(data_result['arr_0'][0]['burst_width'][:].astype(float))
#         print(data_result['arr_0'][0]['burst_width'][:].astype(float))
        dm_offset.append(float(data_result['arr_0'][0]['dm'][0]))
        try:
            width_err.append(data_result['arr_0'][1]['burst_width'][:].astype(float))
            sc_error.append(float(data_result['arr_0'][1]['scattering_timescale'][0]))
            dm_offset_err.append(float(data_result['arr_0'][1]['dm'][0]))
        except:
            width_err.append(float('0'))
            sc_error.append(float('0'))
            dm_offset_err.append(float('0'))
        #width = data_result['arr_0'][0]['burst_width']
# for i in range(len(downsamps)):
#     print(downsamps[i]*2.56e-6, widths[i], width_err[i])
#print(scatterings, "\n")
#print(widths, "\n")
# print(dm_offset,"\n")
# print(width_err,"\n")
# print(sc_error,"\n")
# print(dm_offset_err)

In [15]:
centers = []
peaks = []
uppers = []
lowers = []
bws = []
for result_file in result_files:
    #print(result_file)
    if len(result_file) > 0:
        data_result = np.load(result_file[0], allow_pickle = True)
        try:
            model = data_result['arr_0'][2] ##load the fitburst model
        except:
            model = data_result['arr_0'][1]
        y = np.sum(model, axis=1)
        y_lim = np.max(y)/10
        freq = np.linspace(400,800,1024)
        low = np.min(np.where(y>y_lim))
        lowers.append(float(freq[low]))
        high = np.max(np.where(y>y_lim))
        uppers.append(float(freq[high]))
        center = (freq[low] + freq[high])/2
        centers.append(float(center))
        bw = freq[high]-freq[low]
        #print(center, bw)
        bws.append(float(bw))
        peak = np.argmax(y)
        #print(freq[peak])
        peaks.append(float(freq[peak]))

In [17]:
R3_df_pol = pd.DataFrame(list(zip(event_ids, DMs, downsamps, scatterings, widths, width_err, 
                                 sc_error, dm_offset, dm_offset_err, peaks, bws, centers))
                     ,columns =['Event_id', 'DM_struc', 'Downsample', 'Scattering(s)', 'Widths(s)', 
                               'Width_err(s)','Sc_err(s)','dm_offset','dm_offset_err','Peak_freq','Bandwidth','Center_Freq'])


In [19]:
R3_df_pol.to_csv('R3_bb_pub.csv')